In [ ]:
import os

if os.getcwd().endswith("notebooks"):
    os.chdir("..")
    print("using project root as working dir")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
print(f"using {device} device")

In [ ]:
from dataclasses import asdict
from src.args import gridsearch_args, Args
from datetime import datetime
import seaborn as sns
import pandas as pd

from src.evaluator import Evaluator

experiment = dict(
    key=datetime.now().strftime("%d-%m--%H-%M"),
    reps=1,
    args=[
        Args(),
        Args(),
    ]
)
df_result = pd.DataFrame({})

run_len = len(experiment['args']) * experiment['reps']
for i, args in enumerate(experiment["args"]):
    for rep in range(experiment["reps"]):
        run_index = i * experiment['reps'] + rep
        print(f"running evaluator {run_index}/{run_len}")
        evaluator = Evaluator(
            args=args,
            writer_log_dir=f"runs/{experiment['key']}/s{args.epoch_graph_size}-a{args.epoch_graph_alpha}-b{args.epoch_graph_boredom_pth}-l{args.layers}xs{args.layer_size}--{rep}",
            device=device
        )
        # TODO calc time for training and save to df
        evaluator.train(
            optimizer=torch.optim.Adam(evaluator.net.parameters(), lr=1e-3),
            save_fig=False
        )
        test_loss, test_ap, test_f1 = evaluator.test(
            epoch=args.epochs,
            save_fig=False
        )

        df_result = pd.concat([
            df_result,
            pd.Series({
                "run_index": run_index,
                # run results
                "loss": test_loss,
                "ap": test_ap,
                "f1": test_f1,
                # run args
                **asdict(args),
            }).to_frame().T
        ], ignore_index=True)

    # save results just in case the run fails mid run
    df_result.to_csv(f"{experiment['key']}.csv.zip", index=False, compression=dict(method='zip', archive_name=f"{experiment['key']}.csv"))

In [ ]:
df_result_mean_ap = df_result.groupby(["epoch_graph_size", "epoch_graph_alpha"], as_index=False)["ap"].mean().sort_values(by=['ap'])
print(df_result_mean_ap)

sns.catplot(
    data=df_result_mean_ap,
    col="epoch_graph_size",
    x="epoch_graph_alpha",
    y="ap",
    palette="Spectral",
    kind = "bar"
)

In [ ]:
sns.relplot(
    data=df_result, kind="line",
    hue="epoch_graph_size",
    x="epoch_graph_alpha",
    y="ap",
    errorbar="sd",
    palette="Spectral",
    aspect=2,
)